In [5]:
import os
import gradio as gr
from dotenv import load_dotenv
import ollama
import json


In [6]:
system_message = "Sen FlightAI adında bir havayolu için yardımcı bir asistansın. "
system_message += "Kısa ve nazik cevaplar ver, 1 cümleyi geçme. "
system_message += "Her zaman doğru ol. Eğer cevabı bilmiyorsan, bunu söyle."


In [9]:
def chat(message, history, model="llama3.2"):
    """
    Ollama Llama3.2 için chat fonksiyonu (tek seferlik cevap)
    history: [{"role": "user/assistant", "content": "..."}]
    """
    # Mesajları hazırla
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    
    # Ollama tek seferlik cevap
    response = ollama.chat(model=model, messages=messages)
    
    return response["message"]["content"]

In [10]:
gr.ChatInterface( fn=chat, title="Ollama Chat with Llama3.2", description="Chat with Llama3.2 model using Ollama API", type="messages" ).launch()

* Running on local URL:  http://127.0.0.1:7877
* To create a public link, set `share=True` in `launch()`.


In [11]:
ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")

In [12]:
get_ticket_price("London")  # Example call to the tool

Tool get_ticket_price called for London


'$799'

In [13]:
price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [14]:
tools = [{"type": "function", "function": price_function}]


In [25]:
def chat(message,history,model="llama3.2"):
    """
    Ollama Llama3.2 için chat fonksiyonu (tool destekli)
    history: [{"role": "user/assistant", "content": "..."}]
    """
    # Mesajları hazırla
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    
    # Ollama tool destekli chat
    final_response = ollama.chat(model=model, messages=messages)

    if getattr(final_response,"finis_reason",None)=="tool_calls":
        message_from_model=final_response["messages"]
        tool_response,city = handle_tool_calls(message_from_model)
        messages.append(message_from_model)
        messages.append(tool_response)
        final_response = ollama.chat(model=model, messages=messages)
    return final_response["message"]["content"]
   

In [26]:
def handle_tool_calls(message):
    """
    Tool çağrısını işle ve bilet fiyatını döndür.
    message: model cevabı (dict)
    """
    # message içinden tool çağrısını al (OpenAI mantığı)
    tool_call = message.get("tool_calls", [])[0]  # boşsa hata vermemesi için get
    arguments = json.loads(tool_call["function"]["arguments"])
    city = arguments.get("destination_city")
    
    # Örnek: fiyatı hesapla
    price = get_ticket_price(city)
    
    # Tool cevabı hazırla
    response = {
        "role": "tool",
        "content": json.dumps({"destination_city": city, "price": price}),
        "tool_call_id": tool_call["id"]
    }
    
    return response, city

In [27]:
gr.ChatInterface(fn=chat, title="Ollama Chat with Llama3.2", description="Chat with Llama3.2 model using Ollama API", type="messages").launch()

* Running on local URL:  http://127.0.0.1:7881
* To create a public link, set `share=True` in `launch()`.
